# About this file

This file will read the CSV files created in Part-1 and count the ball possession in different regions of the pitch.
After that it will store the data into new CSV File.

Finally it will merge all the new CSV files.

# 1. Importing Libraries

In [1]:
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import io
import pandas as pd
import random
import numpy as np
import datetime

# 2. Reading File

In [2]:
#Manually upload the file to read data.

# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r1_2.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r3_4.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r5.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r6_7_8.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r9_10_11.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r12_13_14.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r15_16_17.csv"
text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r18_19_20.csv"
# text_file = "E:\\MS DA\\Major Project Nuig\\updated_data_r21.csv"
file_content = pd.read_csv(text_file)
file_content.head(10)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,match_date,match_time,game_id,tracking_area_y,tracking_area_x,pitch_size_y,pitch_size_x,first_period_frame_start,first_period_frame_end,second_period_frame_start,...,teamb,game_id1,frame_no,ball_x,ball_y,ball_p,ball_z,ball_holding_team,ball_status,remove
0,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344191,-172,-2366,0,1.41,A,Dead;,NaN
1,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344192,-171,-2366,0,1.41,A,Dead;,NaN
2,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344193,-169,-2365,0,1.00,A,Dead;,NaN
3,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344194,-168,-2365,0,1.00,A,Dead;,NaN
4,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344195,-166,-2365,0,1.00,A,Dead;,NaN
5,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344196,-163,-2366,0,0.00,A,Dead;,NaN
6,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344197,-161,-2367,0,1.00,A,Dead;,NaN
7,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344198,-160,-2368,0,1.00,A,Dead;,NaN
8,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344199,-160,-2371,0,1.00,A,Dead;,NaN
9,2018-12-22,15:00:00,987762,88.0,111.2,68.0,105.2,1349986,1420588,1443099,...,"Brighton and Hove Albion""",987762,1344200,-159,-2372,0,1.00,A,Dead;,NaN


# 3. Calculating Ball Possession

In [3]:
#Since we are currently looking for the first half of the data hence second half part is commented.
def selecting_data(data):
    #Filtering out the frames which actually has game data.
    first_half = data[data["first_period_frame_start"] <= data["frame_no"]]
    first_half = first_half[first_half["first_period_frame_end"] >= first_half["frame_no"]]

#     second_half = data[data["second_period_frame_start"] <= data["frame_no"]]
#     second_half = second_half[second_half["second_period_frame_end"] >= second_half["frame_no"]]
#     pd.concat([first_half, second_half])
    return first_half

In [4]:
def manual_zones_draw_1(raw_data):
    data = selecting_data(raw_data)
    #Fetch the size of the pitch from dataset.
    pitch_x_len = data.pitch_size_x.iloc[0]
    pitch_y_len = data.pitch_size_y.iloc[0]
    
    starting_point_x = -(pitch_x_len/2) #leftmost point of the pitch
    starting_point_y = -(pitch_y_len/2) #leftmost point of the pitch
    ending_point_x = (pitch_x_len/2) #rightmost point of the pitch
    ending_point_y = (pitch_y_len/2) #rightmost point of the pitch
        
    team_a_zone_1_count = 0
    team_a_zone_2_count = 0
    team_a_zone_3_count = 0
    team_a_zone_4_count = 0
    team_a_zone_5_count = 0
    team_a_zone_6_count = 0
    team_a_zone_7_count = 0
    team_a_zone_8_count = 0
    team_a_zone_9_count = 0
    team_b_zone_1_count = 0
    team_b_zone_2_count = 0
    team_b_zone_3_count = 0
    team_b_zone_4_count = 0
    team_b_zone_5_count = 0
    team_b_zone_6_count = 0
    team_b_zone_7_count = 0
    team_b_zone_8_count = 0
    team_b_zone_9_count = 0
    
    for i in range(len(data)):
        #Check if ball is inside the pitch
        if (starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x and starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y):
            if(data.ball_holding_team.iloc[i] == "H"): #Check if ball is with home team or away team
                if((starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= -17.5) & (starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= -11)):
                    team_a_zone_1_count += 1
                elif((starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= -17.5) & (-11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= 11)):
                    team_a_zone_2_count += 1
                elif((starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= -17.5) & (11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y)):
                    team_a_zone_3_count += 1
                elif((-17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= 17.5) & (starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= -11)):
                    team_a_zone_4_count += 1
                elif((-17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= 17.5) & (-11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= 11)):
                    team_a_zone_5_count += 1
                elif((-17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= 17.5) & (11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y)):
                    team_a_zone_6_count += 1
                elif((17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x) & (starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= -11)):
                    team_a_zone_7_count += 1
                elif((17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x) & (-11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= 11)):
                    team_a_zone_8_count += 1
                elif((17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x) & (11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y)):
                    team_a_zone_9_count += 1
            elif(data.ball_holding_team.iloc[i] == "A"):
                if((17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x) & (starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= -11)):
                    team_b_zone_1_count += 1
                elif((17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x) & (-11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= 11)):
                    team_b_zone_2_count += 1
                elif((17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= ending_point_x) & (11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y)):
                    team_b_zone_3_count += 1
                elif((-17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= 17.5) & (starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= -11)):
                    team_b_zone_4_count += 1
                elif((-17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= 17.5) & (-11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= 11)):
                    team_b_zone_5_count += 1
                elif((-17.5 <= data.ball_x.iloc[i]/int(pitch_x_len) <= 17.5) & (11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y)):
                    team_b_zone_6_count += 1
                elif((starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= -17.5) & (starting_point_y <= data.ball_y.iloc[i]/int(pitch_y_len) <= -11)):
                    team_b_zone_7_count += 1
                elif((starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= -17.5) & (-11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= 11)):
                    team_b_zone_8_count += 1
                elif((starting_point_x <= data.ball_x.iloc[i]/int(pitch_x_len) <= -17.5) & (11 <= data.ball_y.iloc[i]/int(pitch_y_len) <= ending_point_y)):
                    team_b_zone_9_count += 1
    
    zone_counts = [team_a_zone_1_count, team_a_zone_2_count, team_a_zone_3_count, team_a_zone_4_count, team_a_zone_5_count, team_a_zone_6_count, team_a_zone_7_count, team_a_zone_8_count, team_a_zone_9_count, team_b_zone_1_count, team_b_zone_2_count, team_b_zone_3_count, team_b_zone_4_count, team_b_zone_5_count, team_b_zone_6_count, team_b_zone_7_count, team_b_zone_8_count, team_b_zone_9_count]

    return zone_counts

In [5]:
#Grouping Teams on the basis of Game Id
grouped_games = file_content.groupby("game_id")

In [6]:
len(grouped_games.groups.keys())

30

In [7]:
zone_details = []
game_ids = []
#Calculating the ball-possesion on the basis of different zones
for n,gp in grouped_games:
    print(n)
    zone_details.append(manual_zones_draw_1(gp))
    game_ids.append(n)

987762
987763
987764
987765
987766
987767
987768
987769
987770
987771
987772
987773
987774
987775
987776
987777
987778
987779
987780
987781
987782
987783
987784
987785
987786
987787
987788
987789
987790
987791


In [8]:
zone_details[0]

[1229,
 708,
 491,
 2762,
 1280,
 1931,
 1621,
 2106,
 1233,
 3805,
 5215,
 1403,
 3703,
 2245,
 3582,
 1755,
 3500,
 3515]

In [9]:
home_team = []
away_team = []
#Cleaning the Team name data
for n,gp in grouped_games:
    home_team.append(gp.iloc[0]["teama"].split('"')[-1])
    away_team.append(gp.iloc[0]["teamb"].split('"')[0])

In [10]:
#Converting data into Data Frame for futher use
df_zone_details = pd.DataFrame(zone_details, columns = ["team_a_zone_1_count", "team_a_zone_2_count", "team_a_zone_3_count", "team_a_zone_4_count", "team_a_zone_5_count", "team_a_zone_6_count", "team_a_zone_7_count", "team_a_zone_8_count", "team_a_zone_9_count", "team_b_zone_1_count", "team_b_zone_2_count", "team_b_zone_3_count", "team_b_zone_4_count", "team_b_zone_5_count", "team_b_zone_6_count", "team_b_zone_7_count", "team_b_zone_8_count", "team_b_zone_9_count"])
df_zone_details["match_id"] = game_ids
df_zone_details["teama"] = home_team
df_zone_details["teamb"] = away_team

In [11]:
df_zone_details.head()

,team_a_zone_1_count,team_a_zone_2_count,team_a_zone_3_count,team_a_zone_4_count,team_a_zone_5_count,team_a_zone_6_count,team_a_zone_7_count,team_a_zone_8_count,team_a_zone_9_count,team_b_zone_1_count,...,team_b_zone_3_count,team_b_zone_4_count,team_b_zone_5_count,team_b_zone_6_count,team_b_zone_7_count,team_b_zone_8_count,team_b_zone_9_count,match_id,teama,teamb
0,1229,708,491,2762,1280,1931,1621,2106,1233,3805,...,1403,3703,2245,3582,1755,3500,3515,987762,Bournemouth,Brighton and Hove Albion
1,2354,1914,2035,3319,3476,4061,1082,684,1053,1442,...,1358,3000,3085,3360,1476,3007,1601,987763,Arsenal,Burnley
2,920,1562,489,1273,706,630,658,118,731,2318,...,2051,4631,4856,3881,4083,3201,1957,987764,Cardiff City,Manchester United
3,2948,1300,2388,4335,4114,3699,1025,1493,1762,1427,...,700,1583,1023,1254,3564,6563,2377,987765,Chelsea,Leicester City
4,1058,784,991,1796,1283,1815,1622,2686,821,1540,...,2162,3302,5955,4750,2479,3742,3977,987766,Everton,Tottenham Hotspur
5,2061,4456,1256,3418,1896,1735,1069,460,454,2275,...,1472,4876,3261,1832,1259,2802,1329,987767,Huddersfield Town,Southampton
6,621,1112,332,5525,6770,5944,1943,1624,3170,1501,...,2191,3839,4124,1465,872,563,179,987768,Manchester City,Crystal Palace
7,1277,940,1280,2506,2902,2409,2038,2643,1628,1736,...,1216,3536,3644,4482,5254,5498,2201,987769,Newcastle United,Fulham
8,774,1317,482,2066,1084,2480,1803,742,637,4709,...,1737,2917,2836,3881,2475,2537,1149,987770,West Ham United,Watford
9,659,561,893,2374,1793,1991,904,2660,1493,942,...,2542,2682,3711,4519,2467,3719,2071,987771,Wolverhampton Wanderers,Liverpool


# 4. Stroing the counted possession in a file

In [12]:
#Save the file

# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_1_2_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_3_4_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_5_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_6_7_8_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_9_10_11_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_12_13_14_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_15_16_17_zones_count.csv", index = False)
# df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_18_19_20_zones_count.csv", index = False)
df_zone_details.to_csv("E:\\MS DA\\Major Project Nuig\\round_21_zones_count.csv", index = False)

# 5. Merging ball-possessions file into a single CSV

In [13]:
#Storing file in a list
csv_files_list = [
    "E:\\MS DA\\Major Project Nuig\\round_1_2_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\updated_data_r3_4.csv",
    "E:\\MS DA\\Major Project Nuig\\round_5_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\round_6_7_8_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\round_9_10_11_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\round_12_13_14_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\round_15_16_17_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\round_18_19_20_zones_count.csv",
    "E:\\MS DA\\Major Project Nuig\\round_21_zones_count.csv"
]

#Defining the output file
output_file_path = "E:\\MS DA\\Major Project Nuig\\complete_zones_counts.csv"

#Opening the file and over-writing it
with open(output_file_path, "r+") as fout:
    header_written = False

    for file in csv_files_list:
        with open(file) as fin:
            header = fin.readline()
            
            if not header_written: #to remove header other than main one
                fout.write(header)
                header_written = True
                
            for line in fin:
                fout.write(line)